In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import StepLR
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Data augmentation for training data
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Standard transformation for validation data
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
])


# Datasets
train_dataset = torchvision.datasets.ImageFolder(root='../datasets/train', transform=train_transform)
val_dataset = torchvision.datasets.ImageFolder(root='../datasets/val', transform=val_transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=256, shuffle=False)



In [ ]:
class EnhancedCNN(torch.nn.Module):
    def __init__(self):
        super(EnhancedCNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, 3, padding=1)
        self.relu1 = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(32, 64, 3, padding=1)
        self.relu2 = torch.nn.ReLU()
        self.conv3 = torch.nn.Conv2d(64, 128, 3, padding=1)
        self.relu3 = torch.nn.ReLU()
        self.conv4 = torch.nn.Conv2d(128, 256, 3, padding=1)
        self.relu4 = torch.nn.ReLU()
        self.conv5 = torch.nn.Conv2d(256, 512, 3, padding=1)
        self.relu5 = torch.nn.ReLU()
        self.conv6 = torch.nn.Conv2d(512, 1024, 3, padding=1)
        self.relu6 = torch.nn.ReLU()

        self.pool = torch.nn.MaxPool2d(2, 2)
        self.fc1 = None
        self.fc2 = torch.nn.Linear(1024, 2)

        # Dummy forward pass to initialize fc1 with the correct input size
        with torch.no_grad():
            self._initialize_fc1(torch.zeros(1, 3, 224, 224))

    def conv_forward(self, x):
        x = self.pool(self.relu1(self.conv1(x)))
        x = self.pool(self.relu2(self.conv2(x)))
        x = self.pool(self.relu3(self.conv3(x)))
        x = self.pool(self.relu4(self.conv4(x)))
        x = self.pool(self.relu5(self.conv5(x)))
        x = self.pool(self.relu6(self.conv6(x)))
        x = x.view(x.size(0), -1)
        return x

    def forward(self, x):
        if self.fc1 is None:
            self._initialize_fc1(x)
        x = self.conv_forward(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def _initialize_fc1(self, x):
        x = self.conv_forward(x)
        x = x.view(x.size(0), -1)
        n_features = x.size(1)
        self.fc1 = torch.nn.Linear(n_features, 1024)

# Function to run training and validation loop
def train_and_evaluate(model, optimizer, scheduler, criterion, train_loader, val_loader, num_epochs=15, patience=5, min_delta=0.001):
    epoch_data = []
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_accuracy = 100 * correct_train / total_train
        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_accuracy = 100 * correct_val / total_val
        avg_val_loss = val_loss / len(val_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {val_accuracy} %, Average loss: {val_accuracy}')

        if (best_val_loss - avg_val_loss) > min_delta:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f'Early stopping triggered at epoch {epoch + 1}')
            break

        epoch_data.append({
            'Epoch': epoch + 1,
            'Training Loss': avg_train_loss,
            'Training Accuracy': train_accuracy,
            'Validation Loss': avg_val_loss,
            'Validation Accuracy': val_accuracy
        })

        scheduler.step()

    return pd.DataFrame(epoch_data), val_accuracy

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers

# Define the model
model = models.Sequential([
    # Define the input shape in the first layer of the neural network
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(1024, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    # The new FC layer, you can also add dropout layer here
    layers.Dense(1024, activation='relu'),
    
    # Add the final output layer
    layers.Dense(2, activation='softmax')
])

# Now plot the model
tf.keras.utils.plot_model(model, to_file='enhanced_cnn_model.png', show_shapes=True, show_layer_names=True, rankdir='TB')


In [ ]:
learning_rates = [1e-2,1e-4,1e-3]
best_accuracy = 0
best_lr = 0

# Initialize dictionaries to store metrics for each learning rate
all_train_losses = {}
all_train_accuracies = {}
all_val_losses = {}
all_val_accuracies = {}

for lr in learning_rates:
    model = EnhancedCNN().to(device)  
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

    # Train and evaluate the model
    metrics_df, val_accuracy = train_and_evaluate(model, optimizer, scheduler, criterion, train_loader, val_loader)

    # Collect metrics
    all_train_losses[lr] = metrics_df['Training Loss'].tolist()
    all_train_accuracies[lr] = metrics_df['Training Accuracy'].tolist()
    all_val_losses[lr] = metrics_df['Validation Loss'].tolist()
    all_val_accuracies[lr] = metrics_df['Validation Accuracy'].tolist()

    # Update the best accuracy and learning rate if the current one is better
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_lr = lr

# Plotting the results
sns.set()  # Use seaborn's default style for the plots

def plot_metrics(metrics_dict, title):
    plt.figure(figsize=(15, 5))
    for lr, metrics in metrics_dict.items():
        plt.plot(metrics, label=f'lr={lr}')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

# Plot all metrics
plot_metrics(all_train_losses, 'Train Loss')
plot_metrics(all_train_accuracies, 'Train Accuracy')
plot_metrics(all_val_losses, 'Test Loss')
plot_metrics(all_val_accuracies, 'Test Accuracy')


In [ ]:
from PIL import Image

# Define a function to perform the prediction on each image
def predict_class(image_path, model, val_transform, class_to_idx):
    # Load and transform the image
    image = Image.open(image_path).convert('RGB')
    image = val_transform(image)

    # Add a batch dimension
    image = image.unsqueeze(0)

    # Model prediction
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
    
    # Get the class name
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    predicted_class_name = idx_to_class[predicted.item()]
    
    return predicted_class_name

# Assuming val_transform, model, and train_dataset are already defined and available
class_to_idx = train_dataset.class_to_idx

# Loop through each test image and store results
predictions = {}
for i in range(1, 501):  # Assuming images are named from 1.jpg to 500.jpg
    image_path = f'../datasets/test/{i}.jpg'  # Update the path as per your directory structure
    predicted_class_name = predict_class(image_path, model, val_transform, class_to_idx)
    predictions[image_path] = predicted_class_name

# Display the predictions
for key, value in predictions.items():
    print(f"{key}, Predicted Class: {value}")


In [ ]:
checkpoint_name = f'PengYangling/IE4483/model{lr}.pth'
torch.save(model.state_dict(), checkpoint_name)